<a href="https://colab.research.google.com/github/MarcoAQS/seminario-/blob/main/Tarea3_busqueda_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 3. Ejercicio 8

Marco Antonio Quintero Santiago

In [50]:
import numpy as np
import random
import networkx as nx

In [51]:
def cobertura(g, v_prim):
    """
    Función que evalua si un subconjunto de vértices
    (expresado en forma binaria) es una cobertura de
    la gráfica.

    Input:
    - g: Una gráfica (usando networkx)
    - vertices: subconjunto de vértices (binario)

    Returns:
    - True (boolean) si el subconjunto de vértices es una cobertura
    - False (boolean) en otro caso
    """

    for arista in g.edges():
        if v_prim[arista[0]] == 0 and v_prim[arista[1]] == 0:
            return False # Si se encuentra una arista que no está cubierta, se regresa False
        pass # Si todas las aristas están cubiertas, se regresa True

    return True


In [52]:
def funcion_f(s,g):
    if cobertura(g,s):
        return sum(s)
    else: #Penalizamos una solución que no sea factible
        return sum(s) + (len(g.edges())*sum(s))

In [53]:
def genera_grafica_aleatoria(n:int, p:float):
    """
    Función para generar n vertices con una probabilidad p
    de agregar la arista que una dos vertices

    Input:
    - n (int): Numero de nodos de la grafica.
    - p (float): Proba de agregar una arista.

    Returns:
    - g (nx.Graph): La grafica generada.
    """
    g = nx.Graph()
    g.add_nodes_from(range(n))
    for i in range(n):
        for j in range(i+1, n):
            if random.random() < p:
                g.add_edge(i, j)
    return g

## Vecindades

In [65]:
def vecindadC(s, k = 0):
    vecinos = []
    for i in range(len(s)):
        vecino = s.copy()
        vecino[i] = 1 - vecino[i]
        vecinos.append(vecino)

    return vecinos


def vecindadRK(s,k):
    vecinos = []
    for i in range(k):
        vecino = s.copy()
        j = random.randint(0,len(s)-1)
        vecino[j] = 1 - vecino[j]
        vecinos.append(vecino)

    return vecinos

In [145]:
# Funcion para generar una solucion aleatoria
def genera_solucion_aleatoria(g,n,N = vecindadRK, m =1):
    """
    Se generá una solución de tamaño n y se verifica que
    sea una solución factible.


    Input:
    - g (nx.Graph): La gráfica con la que se está trabajando
    - n (int): Longitud de la solución.
    - N (function) : La vecindad a utilizar (Por defecto se
                     utiliza la vecindadRK)
    - m (int) : El número aleatorio de vecinos a generar si se
                utiliza la vecindadRK (por defecto se genera 1)

    Returns:
    sol: Una lista con una solución factible generada al azar.
    """
    sol = [random.randint(0, 1) for i in range(n)]

    #evaluamos
    while not cobertura(g,sol):
        sol = N(sol, m)[0]

    return sol

In [133]:
def busqueda_local(s0, g, N, n = 10, f=funcion_f, m = 0):
    x = s0
    z = f(x,g)

    k = 0

    while k < n:
        vecinos = N(x,m)
        y = min(vecinos, key = lambda x: f(x,g))
        w = f(y,g)

        if w < z and cobertura(g,y):
            x = y
            z = w
            k = 0
        else:
            k += 1

    return x, z

### Instancia 1

In [146]:
random.seed(0)
n1 = 6
g = genera_grafica_aleatoria(n1, 0.6)

for edge in g.edges():
    print(edge)

(0, 3)
(0, 4)
(0, 5)
(1, 2)
(1, 4)
(1, 5)
(2, 3)
(2, 5)
(3, 4)


In [147]:
random.seed(0)
sol0_1 = genera_solucion_aleatoria(g,n1)
sol0_1

[1, 1, 0, 1, 1, 1]

In [148]:
busqueda_local(sol0_1, g, vecindadC)

([0, 1, 0, 1, 1, 1], 4)

In [149]:
busqueda_local(sol0_1, g, vecindadRK, m = 10)

([1, 1, 0, 1, 0, 1], 4)

### Instancia 2

In [139]:
random.seed(1)
n2 = 15
h = genera_grafica_aleatoria(n2, 0.3)

for edge in h.edges():
    print(edge)

(0, 1)
(0, 4)
(0, 9)
(0, 10)
(0, 14)
(1, 4)
(1, 7)
(1, 8)
(1, 12)
(1, 14)
(2, 3)
(2, 6)
(2, 7)
(2, 8)
(2, 10)
(2, 11)
(3, 4)
(3, 7)
(4, 11)
(4, 12)
(5, 6)
(6, 9)
(6, 10)
(7, 8)
(7, 14)
(8, 13)
(9, 11)
(11, 13)
(12, 14)


In [150]:
random.seed(1)
sol0_2 = genera_solucion_aleatoria(h,n2,vecindadRK,m=1)
sol0_2

[1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0]

In [151]:
busqueda_local(sol0_2, h,vecindadC)

([1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0], 9)

In [152]:
busqueda_local(sol0_2, h, vecindadRK,m=10)

([1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0], 9)

In [128]:
cobertura(h,busqueda_local(sol0_2, h, vecindadRK,m=10)[0])

True

### Instancia 3

In [172]:
random.seed(2)
n3 = 25
q = genera_grafica_aleatoria(n3, 0.2)

In [173]:
random.seed(2)
sol0_3 = genera_solucion_aleatoria(q,n3)
sol0_3

[1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [174]:
busqueda_local(sol0_3, q,vecindadC)

([1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 16)

In [176]:
random.seed(1)
busqueda_local(sol0_3, q,vecindadRK)

([1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 18)